In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [43]:
dataset=pd.read_csv('C:/Users/Veenu/Downloads/train/train.csv')

In [44]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
dataset=dataset.dropna()

In [47]:
x=dataset.drop('label',axis=1)
y=dataset['label']

In [48]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [49]:
x.shape

(18285, 4)

In [50]:
y.shape

(18285,)

In [51]:
import tensorflow as tf

In [52]:
tf.__version__

'2.0.0'

In [53]:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [54]:
voc_size=5000

In [55]:
messages=x.copy()

In [56]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [59]:
messages.reset_index(inplace=True)

In [60]:
import nltk
import re
from nltk.corpus import stopwords

In [67]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Veenu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [74]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review = review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [77]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [79]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...    8 1413 4588]
 [   0    0    0 ... 2413 2944 4536]
 [   0    0    0 ... 1880 1096 1314]
 ...
 [   0    0    0 ... 3778 1706 2972]
 [   0    0    0 ...  533 2699  308]
 [   0    0    0 ...   82 1998  610]]


In [80]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1079,
       4163, 3569,  118,  450, 4298,   21,    8, 1413, 4588])

In [82]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrices=['accuracy'])

In [85]:
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [86]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [88]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 20s 2ms/sample - loss: 0.3327 - val_loss: 0.1917
Epoch 2/10
12250/12250 [==============================] - 11s 913us/sample - loss: 0.1392 - val_loss: 0.1941
Epoch 3/10
12250/12250 [==============================] - 12s 1ms/sample - loss: 0.0927 - val_loss: 0.2393
Epoch 4/10
12250/12250 [==============================] - 12s 973us/sample - loss: 0.0606 - val_loss: 0.2519
Epoch 5/10
12250/12250 [==============================] - 12s 990us/sample - loss: 0.0358 - val_loss: 0.3085
Epoch 6/10
12250/12250 [==============================] - 11s 928us/sample - loss: 0.0208 - val_loss: 0.4154
Epoch 7/10
12250/12250 [==============================] - 11s 897us/sample - loss: 0.0107 - val_loss: 0.4964
Epoch 8/10
12250/12250 [==============================] - 11s 911us/sample - loss: 0.0091 - val_loss: 0.3634
Epoch 9/10
12250/12250 [==============================] - 11s 892us/sample - loss: 

In [94]:
y_pred=model.predict_classes(X_test)

In [95]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[3116  303]
 [ 226 2390]]


0.9123446561723281